In [42]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install geocoder
!pip install folium

from bs4 import BeautifulSoup as bsoup
import requests
import pandas as pd
import geocoder
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print('Packages fully loaded and installed.')

     |████████████████████████████████| 92kB 6.4MB/s eta 0:00:011
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0  

In [43]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= bsoup(source, 'lxml')
#print(soup.prettify())
table= soup.find('table',class_ = 'wikitable')
tabledata = table.tbody.text.split('\n\n')
tableitems = []

for rows in tabledata:
    temp  = rows.split('\n')[1:]
    if (temp != []):
       if (temp[1]!='Not assigned'):
        if(temp[2] == 'Not assigned'):
            temp[2] = temp[1]
        tableitems.append(temp)


In [44]:
toronto_df = pd.DataFrame(tableitems)

new_header = toronto_df.iloc[0]
toronto_df = toronto_df[1:]
toronto_df.columns = new_header
toronto_df['Neighborhood'] = toronto_df.groupby(['Postcode', 'Borough'], axis=0)['Neighborhood'].transform(lambda x: ', '.join(x))
toronto_df.drop_duplicates(inplace = True)
toronto_df.reset_index(inplace=True)

In [45]:
toronto_df.shape

(103, 4)

<h2>Part 2</h2>

In [46]:
#import data into pandas df
LatLong = pd.read_csv("http://cocl.us/Geospatial_data")
LatLong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
df_toronto = pd.merge(toronto_df, LatLong, left_on = 'Postcode', right_on ='Postal Code')
df_toronto = df_toronto.drop(['index','Postal Code'], axis = 1)
df_toronto= df_toronto.rename(columns={"Postcode": "PostalCode"})
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


<h2>Part 3</h3>